## 使用metaclass实现一个插件系统
假设我们要开发一个插件系统，所有插件类(e.g AddLayerNormPlugin、RoPEPlugin等)都自动注册到一个中央仓库, 而不需要手动添加

## 什么时候需要设计插件系统
插件系统适合以下场景：

* 需要扩展功能但不想修改核心代码

* 不同用户/场景需要不同功能组合
数据分析工具可能允许用户按需加载数据源插件（Excel、SQL、API等）。

* 希望降低代码耦合度
核心系统与插件通过接口交互，避免硬编码依赖。

* 动态加载和卸载功能
服务器程序可以在运行时加载鉴权插件或日志插件。

In [2]:
class PluginMeta(type):
    registry = {}

    def __new__(cls, name, bases, namespace):
        # 1. 创建新类
        new_cls = super().__new__(cls, name, bases, namespace)

        # 忽略BasePlugin, 只注册子类
        if name != "BasePlugin":
            plugin_name = namespace.get("name", name.lower())
            cls.registry[plugin_name] = new_cls

        return new_cls

class BasePlugin(metaclass=PluginMeta):
    name = "base"

    def forward(self, *args, **kwargs):
        raise NotImplementedError("Subclasses must implement forward")

class AddLayerNormPlugin(BasePlugin):
    name = "addlayernorm"

    def forward(self, x, y, eps=1e-10):
        z = x + y
        return z + 1

print(PluginMeta.registry)


{'addlayernorm': <class '__main__.AddLayerNormPlugin'>}


In [3]:
class RoPEPlugin(BasePlugin):
    name = "rope"

    def forward(self, x, cos, sin):
        return x * cos + x * sin

print(PluginMeta.registry)

{'addlayernorm': <class '__main__.AddLayerNormPlugin'>, 'rope': <class '__main__.RoPEPlugin'>}


In [5]:
rope_plugin_cls = PluginMeta.registry["rope"]
rope_plugin = rope_plugin_cls()
result = rope_plugin.forward(1, 2, 3)
print(result)

5


## 控制类的实例化
比如可以确保类只有一个实例(Singleton)

In [18]:
import warnings

class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        else:
            warnings.warn(f"{cls} already exists, it is singleton")
            
        return cls._instances[cls]

class Log(metaclass=SingletonMeta):
    def __init__(self, name, level):
        self.name = name
        self.level = level

log1 = Log("qwen", "info")
log2 = Log("deepseek", "debug")

class AnotherSingleton(metaclass=SingletonMeta):
    def __init__(self):
        self.num = 5

a = AnotherSingleton()
b = AnotherSingleton()


<ipython-input-18-f1e7e49b176b>:10: UserWarning: <class '__main__.Log'> already exists, it is singleton
  warnings.warn(f"{cls} already exists, it is singleton")
<ipython-input-18-f1e7e49b176b>:10: UserWarning: <class '__main__.AnotherSingleton'> already exists, it is singleton
  warnings.warn(f"{cls} already exists, it is singleton")
